In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
cd "/home/marchesini_id/buckets/b1/datasets"

/home/marchesini_id/buckets/b1/datasets


In [6]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///dmeyf2.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
%%sql
create or replace table competencia_fe as
select
    *
from read_csv_auto('competencia_02_ternaria.csv.gz')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [8]:
%%sql
select * from competencia_fe limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87,CONTINUA
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00,CONTINUA
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31,CONTINUA
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66,CONTINUA
5,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,1,0.00,-5791.17,0.00,7827,12539.39,6,0,797.64,CONTINUA
6,29188490,201901,1,0,1,43,281,-520.87,6116.69,2138.32,...,8,9172.86,-29810.71,0.00,7568,73132.65,41,0,9172.86,CONTINUA
7,29188883,201901,1,0,1,64,296,64862.09,459723.16,988.28,...,8,0.00,-1332.76,0.00,7504,1102.50,1,0,58.65,CONTINUA
8,29189950,201901,1,0,1,56,240,13535.41,41250.39,7158.16,...,1,0.00,-20419.25,66.25,3717,27701.56,19,0,1137.81,CONTINUA
9,29190599,201901,1,0,1,59,296,2244.16,24670.53,113.76,...,22,8727.12,-16718.98,37.05,7137,1388.73,2,0,1219.92,CONTINUA


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
limit 10

In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_02
limit 10

In [ ]:

%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ctrx_quarter
from competencia_02
limit 10

In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
  , max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6
  , min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

In [ ]:
campos = ['mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'mcuentas_saldo', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'mprestamos_personales', 'mprestamos_prendarios', 'mprestamos_hipotecarios', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'minversion1_pesos', 'minversion1_dolares', 'minversion2', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 'mcuenta_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'mpagodeservicios', 'mpagomiscuentas', 'mcajeros_propios_descuentos', 'mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mforex_buy', 'mforex_sell', 'mtransferencias_recibidas', 'mtransferencias_emitidas', 'mextraccion_autoservicio', 'mcheques_depositados', 'mcheques_emitidos', 'mcheques_depositados_rechazados', 'mcheques_emitidos_rechazados', 'matm',  'matm_other', 'ctrx_quarter', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo',  'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 'Visa_madelantodolares', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo']
nuevos_features_regr = ""
nuevos_features_lag =""
for campo in campos:
  nuevos_features_regr += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as ctrx_{campo}_slope_6"
  nuevos_features_lag += f"\n, lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}"
print(nuevos_features_regr)
print(nuevos_features_lag)


In [ ]:
%%sql
select * from competencia_02 limit 10

In [ ]:
%%sql
select cproductos, tcuentas, ccuenta_corriente, mpasivos_margen, mcuenta_corriente_adicional, mcuenta_corriente, ccaja_ahorro, mcaja_ahorro, mcaja_ahorro_adicional, mcaja_ahorro_dolares, cdescubierto_preacordado, mcuentas_saldo, ctarjeta_debito, ctarjeta_debito_transacciones from competencia_02 limit 10 

In [ ]:
%%sql
select numero_de_cliente, foto_mes, ctrx_quarter
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_slope_6
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_annual_slope_6
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_slope_6
, regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mactivos_margen_slope_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mpasivos_margen_slope_6
, regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_adicional_slope_6
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_slope_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_slope_6
, regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_adicional_slope_6
, regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_dolares_slope_6
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as ctrx_mcuentas_saldo_slope_6
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_debito_transacciones_slope_6
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as ctrx_mautoservicio_slope_6
, regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_visa_transacciones_slope_6
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_consumo_slope_6
, regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_master_transacciones_slope_6
, regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_consumo_slope_6
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_personales_slope_6
, regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_prendarios_slope_6
, regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_hipotecarios_slope_6
, regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_dolares_slope_6
, regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_pesos_slope_6
, regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_6 as ctrx_minversion1_pesos_slope_6
, regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_6 as ctrx_minversion1_dolares_slope_6
, regr_slope(minversion2, cliente_antiguedad) over ventana_6 as ctrx_minversion2_slope_6
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as ctrx_cpayroll_trx_slope_6
, regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as ctrx_mpayroll_slope_6
, regr_slope(mpayroll2, cliente_antiguedad) over ventana_6 as ctrx_mpayroll2_slope_6
, regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_6 as ctrx_cpayroll2_trx_slope_6
, regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_debitos_automaticos_slope_6
, regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_visa_debitos_automaticos_slope_6
, regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_master_debitos_automaticos_slope_6
, regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_6 as ctrx_mpagodeservicios_slope_6
, regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_6 as ctrx_mpagomiscuentas_slope_6
, regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mcajeros_propios_descuentos_slope_6
, regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_descuentos_slope_6
, regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_descuentos_slope_6
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_mantenimiento_slope_6
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_otras_slope_6
, regr_slope(mforex_buy, cliente_antiguedad) over ventana_6 as ctrx_mforex_buy_slope_6
, regr_slope(mforex_sell, cliente_antiguedad) over ventana_6 as ctrx_mforex_sell_slope_6
, regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_recibidas_slope_6
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_emitidas_slope_6
, regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_6 as ctrx_mextraccion_autoservicio_slope_6
, regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_slope_6
, regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_slope_6
, regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_rechazados_slope_6
, regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_rechazados_slope_6
, regr_slope(matm, cliente_antiguedad) over ventana_6 as ctrx_matm_slope_6
, regr_slope(matm_other, cliente_antiguedad) over ventana_6 as ctrx_matm_other_slope_6
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_ctrx_quarter_slope_6
, regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldototal_slope_6
, regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldopesos_slope_6
, regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldodolares_slope_6
, regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumospesos_slope_6
, regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumosdolares_slope_6
, regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Master_mlimitecompra_slope_6
, regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantopesos_slope_6
, regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantodolares_slope_6
, regr_slope(Master_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagado_slope_6
, regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagospesos_slope_6
, regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagosdolares_slope_6
, regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumototal_slope_6
, regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_6 as ctrx_Master_cconsumos_slope_6
, regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_6 as ctrx_Master_cadelantosefectivo_slope_6
, regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagominimo_slope_6
, regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldototal_slope_6
, regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldopesos_slope_6
, regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldodolares_slope_6
, regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumospesos_slope_6
, regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumosdolares_slope_6
, regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Visa_mlimitecompra_slope_6
, regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantopesos_slope_6
, regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantodolares_slope_6
, regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagado_slope_6
, regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagospesos_slope_6
, regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagosdolares_slope_6
, regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumototal_slope_6
, regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_6 as ctrx_Visa_cconsumos_slope_6
, regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_6 as ctrx_Visa_cadelantosefectivo_slope_6
, regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagominimo_slope_6
, lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad
, lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
, lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones
, lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen
, lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
, lag(mcuenta_corriente_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional
, lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
, lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
, lag(mcaja_ahorro_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional
, lag(mcaja_ahorro_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares
, lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
, lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones
, lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio
, lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones
, lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
, lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones
, lag(mtarjeta_master_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo
, lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
, lag(mprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios
, lag(mprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios
, lag(mplazo_fijo_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares
, lag(mplazo_fijo_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos
, lag(minversion1_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos
, lag(minversion1_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares
, lag(minversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2
, lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
, lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
, lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
, lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
, lag(mcuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos
, lag(mttarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos
, lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos
, lag(mpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios
, lag(mpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas
, lag(mcajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos
, lag(mtarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos
, lag(mtarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos
, lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
, lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
, lag(mforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy
, lag(mforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell
, lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas
, lag(mtransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas
, lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio
, lag(mcheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados
, lag(mcheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos
, lag(mcheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados
, lag(mcheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados
, lag(matm, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm
, lag(matm_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other
, lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
, lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal
, lag(Master_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos
, lag(Master_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares
, lag(Master_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos
, lag(Master_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares
, lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra
, lag(Master_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos
, lag(Master_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares
, lag(Master_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado
, lag(Master_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos
, lag(Master_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares
, lag(Master_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal
, lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos
, lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo
, lag(Master_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo
, lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal
, lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos
, lag(Visa_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares
, lag(Visa_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos
, lag(Visa_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares
, lag(Visa_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra
, lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos
, lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares
, lag(Visa_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado
, lag(Visa_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos
, lag(Visa_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares
, lag(Visa_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal
, lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos
, lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo
, lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
from competencia_02 
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

In [10]:
%%sql 
create or replace table competencia_final as
select *
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_slope_6
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_annual_slope_6
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_slope_6
, regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mactivos_margen_slope_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mpasivos_margen_slope_6
, regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_adicional_slope_6
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_slope_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_slope_6
, regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_adicional_slope_6
, regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_dolares_slope_6
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as ctrx_mcuentas_saldo_slope_6
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_debito_transacciones_slope_6
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as ctrx_mautoservicio_slope_6
, regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_visa_transacciones_slope_6
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_consumo_slope_6
, regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_master_transacciones_slope_6
, regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_consumo_slope_6
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_personales_slope_6
, regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_prendarios_slope_6
, regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_hipotecarios_slope_6
, regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_dolares_slope_6
, regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_pesos_slope_6
, regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_6 as ctrx_minversion1_pesos_slope_6
, regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_6 as ctrx_minversion1_dolares_slope_6
, regr_slope(minversion2, cliente_antiguedad) over ventana_6 as ctrx_minversion2_slope_6
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as ctrx_cpayroll_trx_slope_6
, regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as ctrx_mpayroll_slope_6
, regr_slope(mpayroll2, cliente_antiguedad) over ventana_6 as ctrx_mpayroll2_slope_6
, regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_6 as ctrx_cpayroll2_trx_slope_6
, regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_debitos_automaticos_slope_6
, regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_visa_debitos_automaticos_slope_6
, regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_master_debitos_automaticos_slope_6
, regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_6 as ctrx_mpagodeservicios_slope_6
, regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_6 as ctrx_mpagomiscuentas_slope_6
, regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mcajeros_propios_descuentos_slope_6
, regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_descuentos_slope_6
, regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_descuentos_slope_6
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_mantenimiento_slope_6
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_otras_slope_6
, regr_slope(mforex_buy, cliente_antiguedad) over ventana_6 as ctrx_mforex_buy_slope_6
, regr_slope(mforex_sell, cliente_antiguedad) over ventana_6 as ctrx_mforex_sell_slope_6
, regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_recibidas_slope_6
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_emitidas_slope_6
, regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_6 as ctrx_mextraccion_autoservicio_slope_6
, regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_slope_6
, regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_slope_6
, regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_rechazados_slope_6
, regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_rechazados_slope_6
, regr_slope(matm, cliente_antiguedad) over ventana_6 as ctrx_matm_slope_6
, regr_slope(matm_other, cliente_antiguedad) over ventana_6 as ctrx_matm_other_slope_6
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_ctrx_quarter_slope_6
, regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldototal_slope_6
, regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldopesos_slope_6
, regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldodolares_slope_6
, regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumospesos_slope_6
, regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumosdolares_slope_6
, regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Master_mlimitecompra_slope_6
, regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantopesos_slope_6
, regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantodolares_slope_6
, regr_slope(Master_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagado_slope_6
, regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagospesos_slope_6
, regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagosdolares_slope_6
, regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumototal_slope_6
, regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_6 as ctrx_Master_cconsumos_slope_6
, regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_6 as ctrx_Master_cadelantosefectivo_slope_6
, regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagominimo_slope_6
, regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldototal_slope_6
, regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldopesos_slope_6
, regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldodolares_slope_6
, regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumospesos_slope_6
, regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumosdolares_slope_6
, regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Visa_mlimitecompra_slope_6
, regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantopesos_slope_6
, regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantodolares_slope_6
, regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagado_slope_6
, regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagospesos_slope_6
, regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagosdolares_slope_6
, regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumototal_slope_6
, regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_6 as ctrx_Visa_cconsumos_slope_6
, regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_6 as ctrx_Visa_cadelantosefectivo_slope_6
, regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagominimo_slope_6
, lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad
, lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
, lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones
, lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen
, lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
, lag(mcuenta_corriente_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional
, lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
, lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
, lag(mcaja_ahorro_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional
, lag(mcaja_ahorro_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares
, lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
, lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones
, lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio
, lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones
, lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
, lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones
, lag(mtarjeta_master_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo
, lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
, lag(mprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios
, lag(mprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios
, lag(mplazo_fijo_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares
, lag(mplazo_fijo_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos
, lag(minversion1_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos
, lag(minversion1_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares
, lag(minversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2
, lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
, lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
, lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
, lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
, lag(mcuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos
, lag(mttarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos
, lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos
, lag(mpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios
, lag(mpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas
, lag(mcajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos
, lag(mtarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos
, lag(mtarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos
, lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
, lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
, lag(mforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy
, lag(mforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell
, lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas
, lag(mtransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas
, lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio
, lag(mcheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados
, lag(mcheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos
, lag(mcheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados
, lag(mcheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados
, lag(matm, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm
, lag(matm_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other
, lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
, lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal
, lag(Master_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos
, lag(Master_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares
, lag(Master_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos
, lag(Master_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares
, lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra
, lag(Master_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos
, lag(Master_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares
, lag(Master_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado
, lag(Master_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos
, lag(Master_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares
, lag(Master_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal
, lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos
, lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo
, lag(Master_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo
, lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal
, lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos
, lag(Visa_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares
, lag(Visa_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos
, lag(Visa_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares
, lag(Visa_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra
, lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos
, lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares
, lag(Visa_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado
, lag(Visa_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos
, lag(Visa_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares
, lag(Visa_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal
, lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos
, lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo
, lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
from competencia_fe 
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [11]:
%%sql
select * from competencia_final limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,lag_1_Visa_mlimitecompra,lag_1_Visa_madelantopesos,lag_1_Visa_madelantodolares,lag_1_Visa_mpagado,lag_1_Visa_mpagospesos,lag_1_Visa_mpagosdolares,lag_1_Visa_mconsumototal,lag_1_Visa_cconsumos,lag_1_Visa_cadelantosefectivo,lag_1_Visa_mpagominimo
0,29198669,201901,1,0,1,39,107,2065.47,13819.58,1426.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29198669,201902,1,0,1,39,108,1963.02,13010.77,1226.40,...,230318.55,0.0,0.0,0.00,-26000.31,14.06,19095.86,16.0,0.0,1466.25
2,29198669,201903,1,0,1,39,109,1566.45,13372.88,562.37,...,243435.62,0.0,0.0,0.00,-33989.65,0.00,22622.58,15.0,0.0,2322.54
3,29198669,201904,1,0,1,39,110,2133.28,15039.80,687.76,...,252796.16,0.0,0.0,33297.56,-41098.92,56.57,22914.84,18.0,0.0,1607.01
4,29198669,201905,1,0,1,39,111,0.00,0.00,0.00,...,281520.00,0.0,0.0,0.00,-33297.56,0.00,18096.69,12.0,0.0,1384.14
5,29198669,201906,1,0,1,39,112,1873.19,17155.41,879.85,...,281520.00,0.0,0.0,0.00,-27910.08,70.36,23742.51,10.0,0.0,1595.28
6,29198669,201907,1,0,1,39,113,905.05,16970.29,888.74,...,281520.00,0.0,0.0,20346.89,-33180.36,0.00,7798.58,13.0,0.0,1231.65
7,29198669,201908,1,0,1,39,114,611.61,16426.10,1076.60,...,281520.00,0.0,0.0,0.00,-20339.03,18.27,9376.05,18.0,0.0,1302.03
8,29198669,201909,1,0,1,39,115,346.99,16250.76,725.17,...,281520.00,0.0,0.0,0.00,-22643.31,0.00,29093.36,20.0,0.0,2533.68
9,29198669,201910,1,0,1,39,116,0.00,0.00,0.00,...,281520.00,0.0,0.0,11420.45,-39853.26,28.14,9617.68,11.0,0.0,680.34


In [12]:
%%sql
copy competencia_final to 'competencia_02.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
